
### 1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>. You have to use data.csv file for this assignment
### 2. Code the model to classify data like below image. You can use any number of units in your Dense layers.

<img src='https://i.imgur.com/33ptOFy.png'>



# <font color='red'> <b>3. Writing Callbacks </b> </font>
## You have to implement the following callbacks
-  Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.Do not use tf.keras.metrics for calculating AUC and F1 score.

- Save your model at every epoch if your validation accuracy is improved from previous epoch. 

- You have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
- If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

- You have to stop the training if your validation accuracy is not increased in last 2 epochs.

- Use tensorboard for every model and analyse your scalar plots and histograms. (you need to upload the screenshots and write the observations for each model for evaluation)



<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [22]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn
from tensorflow.keras.callbacks import LearningRateScheduler

In [23]:
df = pd.read_csv('data.csv')

In [25]:
y = df['label'].values
x = df[['f1','f2']].values

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, stratify=y)

In [27]:
Y_train = tf.keras.utils.to_categorical(y_train, 2) 
Y_test = tf.keras.utils.to_categorical(y_test, 2)

In [28]:
import numpy as np
from sklearn.metrics import f1_score,auc,roc_curve
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import ReduceLROnPlateau


def scheduler(epoch, lr):
    
    if (epoch % 3 ==0):
        print('entering scheduler',0.95 * lr)
        return 0.95 * lr
    else:
        return lr 
        
  
class CustomFunction(tf.keras.callbacks.Callback):
    def __init__(self,validation_data):
      self.x_test = validation_data[0]
      self.y_test= validation_data[1]
        
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [ val_acc]
        self.history={'val_accuracy': []}
        
    def on_epoch_end(self,epoch,logs={},):
        try:
            loss = logs.get('loss')
            if loss is not None:
                if np.isnan(loss) or np.isinf(loss):
                    print("Invalid loss and terminated at epoch {}".format(epoch))
                    self.model.stop_training = True
            if logs.get('val_accuracy', -1) != -1:
                self.history['val_accuracy'].append(logs.get('val_accuracy'))
            if(epoch != 0 and self.history['val_accuracy'][epoch] > self.history['val_accuracy'][epoch-1]):
                print('\nVal Accuracy Improved\n')
                model_json = self.model.to_json()
                with open("model_save/model-epoch{}-val_accuracy-{}.json".format(epoch,logs.get('val_accuracy')), "w") as json_file:
                    json_file.write(model_json)

                # serialize weights to HDF5
                model.save_weights("model_save/weights-epoch{}-val_accuracy-{}.hdf5".format(epoch,logs.get('val_accuracy')))
                print("Saved model to disk")
            elif(epoch != 0 and self.history['val_accuracy'][epoch] < self.history['val_accuracy'][epoch-1]):
                self.model.optimizer.lr = self.model.optimizer.lr * 0.9
                print('learning rate reduced')
            y_pred= self.model.predict(self.x_test)    
            y_label_pred=np.argmax(y_pred,axis=1)
           
            f1_s = f1_score(self.y_test,y_label_pred,average='micro')
            fpr, tpr, thresholds = roc_curve(self.y_test,y_label_pred)
            auc_score = auc(fpr, tpr)
            #Ideally Micro and Macro F1 Score is same as there is no class imbalance
            print('Micro F1 Score: ',f1_s)
            print('AUC Score: ',auc_score)
        except:
            print('Exception')
        
        
monitor_function =CustomFunction(validation_data=[X_test,y_test])  
learningratescheduler = LearningRateScheduler(scheduler)
earlystop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)




In [29]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=1, seed=30))(input_layer)
#Dense hidden layer
layer2 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=1, seed=30))(layer1)
#Dense hidden layer
layer3 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=1, seed=30))(layer2)
#Dense hidden layer
layer4 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=1, seed=30))(layer3)
#Dense hidden layer
layer5 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=1, seed=30))(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=1, seed=30))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
model.compile(optimizer=tf.keras.optimizers.experimental.SGD(momentum=0.6),
              loss='categorical_crossentropy',metrics=['accuracy'])

In [30]:
model.fit(X_train,Y_train,epochs=10, validation_data=(X_test,Y_test), batch_size=20, callbacks=[monitor_function,learningratescheduler,earlystop])


entering scheduler 0.009499999787658453
Epoch 1/10
207/207 [==============================] - 0s 706us/steps: 0.7321 - accuracy: 
Micro F1 Score:  0.46424242424242423
AUC Score:  0.46424242424242423
670/670 [==============================] - 2s 2ms/step - loss: 0.7294 - accuracy: 0.5057 - val_loss: 0.7069 - val_accuracy: 0.4642 - lr: 0.0095
Epoch 2/10
622/670 [==========================>...] - ETA: 0s - loss: 0.7001 - accuracy: 0.5129
Val Accuracy Improved

Saved model to disk
207/207 [==============================] - 0s 621us/step
Micro F1 Score:  0.5037878787878788
AUC Score:  0.5037878787878788
670/670 [==============================] - 1s 2ms/step - loss: 0.7000 - accuracy: 0.5131 - val_loss: 0.6987 - val_accuracy: 0.5038 - lr: 0.0095
Epoch 3/10
207/207 [==============================] - 0s 613us/steps: 0.7006 - accuracy: 
Micro F1 Score:  0.5037878787878788
AUC Score:  0.5037878787878788
670/670 [==============================] - 1s 1ms/step - loss: 0.7004 - accuracy: 0.5049 - va

In [38]:
np.exp(-1) * 0.01

0.0036787944117144234

<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [31]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(seed=30))(input_layer)
#Dense hidden layer
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(seed=30))(layer1)
#Dense hidden layer
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(seed=30))(layer2)
#Dense hidden layer
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(seed=30))(layer3)
#Dense hidden layer
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(seed=30))(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
model.compile(optimizer=tf.keras.optimizers.experimental.SGD(momentum=0.6),
              loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
model.fit(X_train,Y_train,epochs=10, validation_data=(X_test,Y_test), batch_size=20, callbacks=[monitor_function,learningratescheduler,earlystop])

entering scheduler 0.009499999787658453
Epoch 1/10
207/207 [==============================] - 0s 680us/steps: 0.6934 - accuracy: 0
Micro F1 Score:  0.5
AUC Score:  0.5
670/670 [==============================] - 2s 2ms/step - loss: 0.6934 - accuracy: 0.4904 - val_loss: 0.6932 - val_accuracy: 0.5000 - lr: 0.0095
Epoch 2/10
207/207 [==============================] - 0s 624us/steps: 0.6933 - accuracy: 
Micro F1 Score:  0.5
AUC Score:  0.5
670/670 [==============================] - 1s 2ms/step - loss: 0.6933 - accuracy: 0.5015 - val_loss: 0.6932 - val_accuracy: 0.5000 - lr: 0.0095
Epoch 3/10
207/207 [==============================] - 0s 621us/steps: 0.6934 - accuracy: 
Micro F1 Score:  0.5
AUC Score:  0.5
670/670 [==============================] - 1s 1ms/step - loss: 0.6934 - accuracy: 0.4946 - val_loss: 0.6933 - val_accuracy: 0.5000 - lr: 0.0095
Epoch 3: early stopping


<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [34]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
#Dense hidden layer
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
#Dense hidden layer
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer2)
#Dense hidden layer
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer3)
#Dense hidden layer
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.he_uniform())(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
model.compile(optimizer=tf.keras.optimizers.experimental.SGD(momentum=0.6),
              loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=10, validation_data=(X_test,Y_test), batch_size=20, callbacks=[monitor_function,learningratescheduler,earlystop])


entering scheduler 0.009499999787658453
Epoch 1/10
207/207 [==============================] - 0s 772us/steps: 0.6452 - accuracy:
Micro F1 Score:  0.6666666666666666
AUC Score:  0.6666666666666666
670/670 [==============================] - 2s 2ms/step - loss: 0.6448 - accuracy: 0.6242 - val_loss: 0.6104 - val_accuracy: 0.6667 - lr: 0.0095
Epoch 2/10
207/207 [==============================] - 0s 612us/step
Micro F1 Score:  0.6656060606060606
AUC Score:  0.6656060606060605
670/670 [==============================] - 1s 2ms/step - loss: 0.6108 - accuracy: 0.6636 - val_loss: 0.6067 - val_accuracy: 0.6656 - lr: 0.0085
Epoch 3/10
207/207 [==============================] - 0s 609us/step
Micro F1 Score:  0.6615151515151515
AUC Score:  0.6615151515151516
670/670 [==============================] - 1s 2ms/step - loss: 0.6063 - accuracy: 0.6672 - val_loss: 0.6126 - val_accuracy: 0.6615 - lr: 0.0077
Epoch 3: early stopping





<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

# Note 
Make sure that you are plotting tensorboard plots either in your notebook or you can try to create a pdf file with all the tensorboard screenshots.Please write your analysis of tensorboard results for each model.
